# Numerical Computation

Machine Learning involves numerical computations that are used to solve mathematical problems via an iterative process that updates the estimates of the solution until some convergence criterion is reached. This is contrary to analytically determining a formula to provide a symbol expression for the correct solution which is a common approach in both engineering and mathematics. Common numerical operations used in ML are:

1. Optimization
2. Solving a system of linear equations

## 4.1 Overflow and Underflow

On a computer, we use a finite number of bit patterns (due to a finite amount of available memory) to represent infinitely many real numbers. This causes us to make approximations that can lead to rounding errors and can even cause a theoretically proven algorithm to fail in practice.

__Underflow__ is one such form of rounding error and occurs when a number near zero gets rounded off to zero. This can cause operations such as "divide by" or "log" to fail since both operations are undefined for 0.

__Overflow__ on the other hand occurs when numbers with large magnitudes are approximated as $\inf$ or $-\inf$. 

The __softmax function__ $softmax(\pmb{x})_i = \dfrac{\exp(x_i)}{\sum_{j = 1}^{n}\exp(x_j)}$ for example, must be stabilized against underflow and overflow. Let's take a look at this example:

In [23]:
import math
def softmax(x, i):
    return math.exp(x[i]) / sum(map(lambda v: math.exp(v), x))

xs = [[1] * 10, [math.pow(2, 32)] * 10, [-math.pow(2, 32)] * 10]

# softmax without exponent
for x in xs:
    try: 
        print(softmax(x, 1))
    except OverflowError:
        print(f'Overflow error for an array with all values = {x[0]}')
    except ZeroDivisionError:
        print(f'Divison By Zero error for an array with all values = {x[0]}')

# Normalize the values in x before taking the exponent
for x in xs:
    try:
        m = max(x)
        zs = list(map(lambda v: v - m, x))
        print(softmax(zs, 1))
    except OverflowError:
        print(f'Overflow error for an array with all values = {x[0]}')
    except ZeroDivisonError:
        print(f'Divison By Zero error for an array with all values = {x[0]}')

0.1
Overflow error for an array with all values = 4294967296.0
Divison By Zero error for an array with all values = -4294967296.0
0.1
0.1
0.1


We can clearly see from the the example above that when $\forall i, x_i = c$ and x has length n then we expect the value of the softmax to be $\dfrac{1}{n}$. But that is not what happens when c is very large or small. To maneuver against this what we do is modify each element in the array, xs using the following operation $\forall i, z_i = x_i - max_j x_j$ and then use the resulting zs to calculate softmax. This maneuvering does not alter the expected output and also allows us to tackle the issues of overflow and division by zero.

However, if our numerator $x_i$ is a large negative number and all the other numbers in the list of values are positive. The numerator of softmax might get round off to zero resulting in underflow. Moreover, if we take $\log(softmax(\pmb{x})$ then we would end up with $-\inf$. But to counteract this we make use of log rules in evaluating $\log(softmax(x)_i)$. We thus present a completely stabilized version of $\log(softmax(x)_i)$ below:

$$
\log(softmax(x)_i) = \log\left(\dfrac{\exp(x_i)}{\sum_{j = 1}^{n}\exp(x_j)}\right) \\
= \log\left(\dfrac{\exp(x_i - b)\exp(b)}{\sum_{j = 1}^{n}\exp(x_j - b)\exp(b)}\right) \\ 
= \log\left(\exp(x_i - b)) - \log(\sum_j \exp(x_j - b)\right) \\
= x_i - \log\left(\sum_j \exp(x_j - b)\right)
$$

## 4.2 Poor Conditioning

Conditioning refers to how rapidly a function changes with respect to small changes in its input. When a function changes rapidly with small changes to its input it can be problematic for scientific computations because of rounding errors. 

Let's consider $f(\pmb x) = A^{-1} \pmb{x}$ assuming $A$ has an eigenvalue decomposition. The __condition number__ for $A$ is:

$$
\max_{i, j} \left|\dfrac{\lambda_i}{\lambda_j}\right|
$$

which is the ratio of the largest and smallest eigenvalues. When this number is large matrix inversion is particularly sensitive to error in inputs. This sensitivity is intrinsic to the matrix itself not the result of rounding error during matrix inversion. Poorly conditioned matrices amplify pre-existing errors when we multiply by the true matrix inverse.


## 4.3 Gradient-Based Optimization

Most deep learning algorithms involve optimization of some sort. Optimization refers to the task of minimizing or maximizing some function $f(x)$ by changing $x$. Here $f$ is the __objective function__ or __criterion__. In deep learning we frame problems as minimizing $f(x)$. During minimization $f$ might also be called the __cost function__, __loss function__ or __error function__. $x^*$ is the value that minimizes the function $f$, so $x^* = arg min f(x)$.

Let's say $y = f(x)$. The __derivative__ of this function is denoted by $f'(x)$ or as $\dfrac{dy}{dx}$. The derivative gives the slow of $f$ at $x$. It specifies how to scale a small change in the input to obtain a corresponding change in the output: $f(x + \epsilon) \approx f(x) + \epsilon f'(x)$.

The derivative is used for minimizing a function because it tells us how to change $x$ to make a small improvement in $y$. We can decrease $f(x)$ by moving $x$ in small steps in the direction with the opposite sign to the derivative. This technique is called __gradient descent__.

TODO: Add picture 4.1 from book here.

When $f'(x) = 0$ the derivative gives us no information about the direction we should move in. These points where the derivative is 0 are known as __critical points__ or __stationary points__. We can classify them as:

1. Local minimum - a point where $f(x)$ is lower than all neighboring points. 
2. Local maximum - a points where $f(x)$ is higher than all neighboring points.
3. Saddle Points - a point where $f(x)$ is neither a maximum or a minimum.

TODO: Add picture 4.2 here.

A point that obtains the lowest value of $f(x)$ across all possible $x$ is called the __global minimum__. A local minimum does not have to be a global minimum. In deep learning we usually settle for finding a value that is very low but not necessarily  minimal in any sense due to the complex nature of functions being minimized.

We often minimize function that are defined as $f: \mathbb{R}^n \implies \mathbb{R}$. For minimization to make sense the output must be a scalar. In case, of multiple inputs we must take __partial derivatives__. The partial derivative $\dfrac{\partial f(\pmb{x})}{\partial x_i}$ measures what direction we should move $x_i$ in to be able to increase $f$ all other variables staying the same. The gradient of a function is denoted as $\bigtriangledown_{\pmb{x}}f(\pmb{x})$. 


